In [1]:
import glob
import rasterio
from pathlib import Path
import os
import reverse_geocoder as rg
import pyproj
import geopandas as gpd
import sys

In [2]:
if Path.cwd().name != 'src':
    os.chdir('/home/vaschetti/maxarSrc/src')

In [3]:
from my_functions import build
events_names = build.get_all_events()

In [34]:
import pandas as pd

class Mosaic:
    def __init__(self,
                 name,
                 event_name,
                 path_mosaic_metatada = '/home/vaschetti/maxarSrc/metadata/from_github_maxar_metadata/datasets',
                 ):
        
        #Paths
        self.path_mosaic_metatada = path_mosaic_metatada

        #Mosaic
        self.name = name
        self.event_name = event_name
        self.bbox, _ = build.get_mosaic_bbox(self.event_name,
                                          self.name,
                                          self.path_mosaic_metatada,
                                          extra_mt=1000)
        
        self.road_gdf = None
        self.build_gdf = None
        
        self.tiles_path = []

        #Event
        
        self.event_road_gdf = None

    def set_event_road_gdf(self, gdf):
        self.event_road_gdf = gdf

    def set_road_gdf(self):
        if self.event_road_gdf is None:
            raise ValueError('event_road_gdf is None')
        self.road_gdf = build.filter_gdf_w_bbox(self.event_road_gdf, self.bbox)
    
    def set_build_gdf(self, buildings_dataset_links_path):
        qk_hits = build.intersecting_qks(*self.bbox)
        self.build_gdf = build.qk_building_gdf(qk_hits, csv_path=buildings_dataset_links_path)
        
    def add_tile(self, tile):
        self.tiles.append(tile)

    def remove_tile(self, tile):
        self.tiles.remove(tile)

    def get_tiles(self):
        return self.tiles
    
    def __str__(self) -> str:
        return self.name

In [31]:
from my_functions import build
import time
import pandas as pd

class Event:
    def __init__(self,
                 name,
                 when = 'pre',
                 maxar_root = '/mnt/data2/vaschetti_data/maxar',
                 ):
        
        #Paths
        self.maxar_root = maxar_root
        self.buildings_ds_links_path = '/home/vaschetti/maxarSrc/metadata/buildings_dataset_links.csv'
        
        #Event
        self.name = name
        self.when = when
        self.region_name = build.get_region_name(self.name)
        self.bbox = build.get_event_bbox(self.name, extra_mt=1000) #TODO può essere ottimizzata sfruttando i mosaici
        self.all_mosaics_names = build.get_mosaics_names(self.name, self.maxar_root, self.when)
        
        
        #Roads
        self.road_gdf = None

        #Mosaics
        self.mosaics = {} #TODO decidere se lista o dict

        #Init mosaics
        for m_name in self.all_mosaics_names:
            self.mosaics[m_name] = Mosaic(m_name, self.name)

    #Roads methods
    def set_road_gdf(self): #set road_gdf for the event
        region_road_gdf = build.get_region_road_gdf(self.region_name)
        self.road_gdf = build.filter_gdf_w_bbox(region_road_gdf, self.bbox)


    def set_road_gdf_in_mos(self, mosaic_name): #set road_gdf for the mosaic
        if self.road_gdf is None:
            self.set_road_gdf()

        self.mosaics[mosaic_name].set_event_road_gdf(self.road_gdf)
        self.mosaics[mosaic_name].set_road_gdf()

    def set_roads_gdf_all_mos(self): #set road_gdf for all the mosaics
        if self.road_gdf is None:
            self.set_road_gdf()

        for mosaic_name, mosaic in self.mosaics.items():
            if mosaic.road_gdf is None:
                self.set_road_gdf_in_mos(mosaic_name)
    
    #Buildings methods
    def set_build_gdf_in_mos(self, mosaic_name):
        self.mosaics[mosaic_name].set_build_gdf(self.buildings_ds_links_path)

    def set_build_gdf_all_mos(self):
        for mosaic_name, mosaic in self.mosaics.items():
            if mosaic.build_gdf is None:
                self.set_build_gdf_in_mos(mosaic_name)

    def get_mosaic(self, mosaic_name):
        return self.mosaics[mosaic_name]

In [32]:
evento = Event('Gambia-flooding-8-11-2022')
all_mosaics_names = evento.all_mosaics_names
print(all_mosaics_names)
m0 = evento.mosaics[all_mosaics_names[0]]
m0.bbox

['10300100CFC9A500', '105001002BD68F00', '1040010073D77D00']


((-16.70926555697382, 13.323422751465731),
 (-16.564347201396977, 13.498854689055355))

In [33]:
evento.init_buildings()

Found 3 links matching: [33303200, 3330320, 333032]
Found 3 links matching: [33303200, 3330320, 333032]
Found 3 links matching: [33303200, 3330320, 333032]


NameError: name 'building_links_df' is not defined

In [12]:
evento.init_roads()

time per leggere il df: 8.127062797546387
get_region_road_gdf: 108.75397634506226


In [16]:
evento.mosaics[2].road_gdf

,country,geometry
158659,GMB,"LINESTRING (-16.68202 13.41127, -16.68201 13.4..."
158664,GMB,"LINESTRING (-16.68379 13.46848, -16.68323 13.4..."
158669,GMB,"LINESTRING (-16.69678 13.36874, -16.69712 13.3..."
158689,GMB,"LINESTRING (-16.68736 13.44269, -16.68723 13.4..."
158694,GMB,"LINESTRING (-16.68226 13.39987, -16.68224 13.3..."
...,...,...
504430,GMB,"LINESTRING (-16.72385 13.40120, -16.72470 13.4..."
504460,GMB,"LINESTRING (-16.69546 13.40399, -16.69566 13.4..."
504475,GMB,"LINESTRING (-16.73466 13.41199, -16.73617 13.4..."
504485,GMB,"LINESTRING (-16.69505 13.36657, -16.69382 13.3..."


In [ ]:
m0 = evento.mosaics[0]
m0.bbox

((-16.70926555697382, 13.323422751465731),
 (-16.564347201396977, 13.498854689055355))